## Importation des packages

In [72]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing 
from sklearn.model_selection import train_test_split

from sklearn.tree import export_graphviz
import pydot

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import scale
import operator
import sklearn.metrics as metrics
 

## Random forest avec la variable date et température

In [9]:
enedis_PACA_date=pd.read_csv("Données/data_one_hot_encoded/data_one_hot_encoded_jour_6kva_PACA.csv",sep=",")
enedis_IDF_date=pd.read_csv("Données/data_one_hot_encoded/data_one_hot_encoded_jour_6kva_IDF.csv",sep=",")
enedis_HDF_date=pd.read_csv("Données/data_one_hot_encoded/data_one_hot_encoded_jour_6kva_HDF.csv",sep=",")


### On enlève les valeurs manquantes

enedis_IDF_date.dropna(inplace=True)
enedis_PACA_date.dropna(inplace=True)
enedis_HDF_date.dropna(inplace=True)

In [11]:
def decoupe_data_date(bdd_region):

    
    X=bdd_region.drop(["conso_totale_moyenne","date","total_energie_soutiree_wh","region_x","indice"],axis=1)
    Y=bdd_region[["conso_totale_moyenne"]]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
    return (X,Y, X_train, X_test, Y_train, Y_test)

Découpage des dataset (apprentissage/test):

In [79]:
## Création des échantillons d'apprentissages et de tests

X_IDF_date,Y_IDF_date , X_train_IDF_date, X_test_IDF_date, Y_train_IDF_date, Y_test_IDF_date =decoupe_data_date(enedis_IDF_date)
X_PACA_date,Y_PACA_date , X_train_PACA_date, X_test_PACA_date, Y_train_PACA_date, Y_test_PACA_date =decoupe_data_date(enedis_PACA_date)
X_HDF_date,Y_HDF_date , X_train_HDF_date, X_test_HDF_date, Y_train_HDF_date, Y_test_HDF_date =decoupe_data_date(enedis_HDF_date)

### Modèle de random Forest :

In [14]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision treesrf = RandomForestRegressor(n_estimators = 1000, random_state = 42)



In [81]:
def genere_arbre(X_train, Y_train,X_test, Y_test):
 #   max_depth_arbre=caract_arbre(X_train, Y_train,X_test, Y_test)
    #instanciation
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, max_depth=4)

    #training
    rf.fit(X_train,Y_train)
    prediction = rf.predict( X_test)
    #calcule de précision
    print("Score de test :",rf.score( X_test, Y_test))
   
     # Calculate the absolute errors
    errors = abs(prediction - Y_test.conso_totale_moyenne.to_numpy())
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'kWh.')
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / Y_test.conso_totale_moyenne.to_numpy())
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    
    return(rf)

In [82]:
modele_IDF_date=genere_arbre(X_train_IDF_date, Y_train_IDF_date,X_test_IDF_date,Y_test_IDF_date)


C:\Users\elelo\AppData\Local\Temp/ipykernel_15608/942165656.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,Y_train)


Score de test : 0.6478652739660087
Mean Absolute Error: 838.41 kWh.
Accuracy: 83.2 %.


In [83]:
modele_HDF_date=genere_arbre(X_train_HDF_date, Y_train_HDF_date,X_test_HDF_date,Y_test_HDF_date)

C:\Users\elelo\AppData\Local\Temp/ipykernel_15608/942165656.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,Y_train)


Score de test : 0.7324996617053627
Mean Absolute Error: 887.02 kWh.
Accuracy: 86.84 %.


In [84]:
modele_PACA_date=genere_arbre(X_train_PACA_date, Y_train_PACA_date,X_test_PACA_date,Y_test_PACA_date)

C:\Users\elelo\AppData\Local\Temp/ipykernel_15608/942165656.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,Y_train)


Score de test : 0.6310973054007067
Mean Absolute Error: 820.81 kWh.
Accuracy: 85.35 %.


### Visualiser l'arbre : (en cours)

In [55]:
feature_list = list(X_IDF_date.columns)

In [73]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = modele_IDF_date.estimators_[5]
# Import tools needed for visualization

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

FileNotFoundError: [WinError 2] "dot" not found in path.

necessité d'une baseline ??? (erreur de référence, par ex conso moyenne pour montrer que c'est significatif)